# Bag of Visual Words
algorithm
1. 全画像をCNN(VGG16)に入力し，局所特徴量(include_top=False)の抽出
2. 全ての局所特徴量をk-meansによってカテゴライズ，ヒストグラムのビンとする
3. ビンを基に，全画像に対してBag of Visual Wordsのヒストグラムを作り，画像の特徴ベクトルを作成

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.preprocessing import image
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import sys
import cv2
import os
import shutil
import networkx as nx
import matplotlib.pyplot as plt 

In [ ]:
DATA_DIR = '../data/'
VIDEOS_DIR = '../data/video/'                        # The place to put the video
TARGET_IMAGES_DIR = '../data/images/target/'         # The place to put the images which you want to execute clustering
CLUSTERED_IMAGES_DIR = '../data/images/cycle/'   # The place to put the images which are clustered
IMAGE_LABEL_FILE ='image_label_cycle.csv'                  # Image name and its label

In [ ]:
DG = nx.DiGraph()

In [ ]:
clusters=50 #k-means
col=7*7 # image
channels=512 #image
k_ctr=4 #k-NN

In [ ]:
def __feature_extraction(model, img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # resize
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)  # add a dimention of samples
    x = preprocess_input(x)  # RGB 2 BGR and zero-centering by mean pixel based on the position of channels

    feat = model.predict(x)  # Get image features ->7x7x512

    return feat

In [ ]:
print('Label images...')
# Load a model
model = VGG16(weights='imagenet', include_top=False)
# Get images
images = [f for f in os.listdir(TARGET_IMAGES_DIR) if f[-4:] in ['.png', '.jpg']]
assert(len(images)>0)

X = []
X_vector = []
for i in range(len(images)):
    # Extract image features
    feat = __feature_extraction(model, TARGET_IMAGES_DIR+images[i])
    X.append(feat)
# Clustering images by k-means++
X = np.array(X)
X = X.reshape([num_img*col, channels])
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(X)
centr = np.array(kmeans.cluster_centers_)

各画像の特徴ベクトルの作成 (Bag of Visual Words)

In [ ]:
for i in range(num_img-1):
    X_hist = [0]*col
    for j in range(col-1):
        dist = [0]*col
        for k in range(clusters-1):
            diff = X[i*col+j,] - centr[k,]
            dist[k] = np.linalg.norm(diff)
        X_hist[dist.index(min(dist))] += 1
    X_vector.append(X_hist)
X_vector = np.array(X_vector) #feature vector
print(np.shape(X_vector)) #(284,49)? (285,49)
DG.add_nodes_from(range(1, len(X_vector))) #make visual k-NN Graph

Bag of Visual Wordsのヒストグラムの画像作成

In [ ]:
df = pd.DataFrame(X_vector)
df = df.transpose()
for i in range(1, len(X_vector)):
    plt.figure()
    df[i-1].plot.bar()
    plt.savefig("../data/hist/hist"+str(i)+".jpg")
    plt.close("all")